In [ ]:
import customtkinter as ctk
from tkinter import filedialog, messagebox, simpledialog
from PIL import Image, ImageTk
import pydicom
import os
from datetime import datetime

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("dark-blue")

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []
selected_cases = []
check_vars = []

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    selected_cases.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    temp_cases = {}
    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            key = (name, date_obj.date(), getattr(ds, "StudyID", ""), getattr(ds, "AccessionNumber", ""))
            if key not in temp_cases:
                img = None
                if 'PixelData' in ds:
                    img_pil = Image.fromarray(ds.pixel_array)
                    img_pil.thumbnail((400, 400))
                    img = ImageTk.PhotoImage(img_pil)

                data_dict = {
                    "Name": name,
                    "Date": date_obj,
                    "CTDIvol": ctdi,
                    "DLP": dlp,
                    "mSv": msv,
                    "Sex": getattr(ds, "PatientSex", ""),
                    "DOB": getattr(ds, "PatientBirthDate", ""),
                    "StudyID": getattr(ds, "StudyID", ""),
                    "Accession": getattr(ds, "AccessionNumber", ""),
                    "Image": img,
                    "Path": path,
                    "Modality": getattr(ds, "Modality", "Unknown"),
                    "Dataset": ds
                }
                temp_cases[key] = data_dict

                hl7_msg = convert_to_hl7(ds, msv)
                hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}_{data_dict['StudyID']}.hl7"
                with open(hl7_filename, "w") as f:
                    f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    all_data.extend(temp_cases.values())
    display_text_data()

def display_text_data():
    for widget in content_frame.winfo_children():
        widget.destroy()

    filtered = []
    name_filter = name_filter_var.get().strip().lower()
    date_filter = date_filter_var.get().strip()

    for data in all_data:
        match_name = name_filter in data["Name"].lower()
        match_date = date_filter in data["Date"].strftime("%Y-%m-%d")
        if match_name and match_date:
            filtered.append(data)

    # نجمع بيانات المرضى حسب الاسم لتسهيل العمليات على كل مريض
    patient_records = {}
    for data in all_data:
        name = data["Name"]
        dose = data["mSv"]
        date = data["Date"]
        if name not in patient_records:
            patient_records[name] = []
        patient_records[name].append((date, dose))

    # نحسب الجرعة التراكمية حسب المطلوب: 
    # أول فحص تراكم الجرعة صفر، تاني فحص = أول + تاني، تالت فحص = تاني + تالت، وهكذا.
    accumulated_dose_dict = {}

    for name in patient_records:
        records = patient_records[name]
        records.sort(key=lambda x: x[0])  # ترتيب حسب التاريخ

        for i, (current_date, current_dose) in enumerate(records):
            if i == 0:
                # أول فحص تراكم الجرعة صفر
                accumulated_dose_dict[(name, current_date)] = 0
            else:
                # الجرعة التراكمية = الجرعة الحالية + الجرعة السابقة فقط
                previous_dose = records[i - 1][1]
                accumulated_dose_dict[(name, current_date)] = round(current_dose + previous_dose, 2)

    # نحسب الجرعة لكل سنة زمنية حقيقية لكل مريض (365 يوم من أول فحص)
    doses_per_year_real = {}

    for name in patient_records:
        records = sorted(patient_records[name], key=lambda x: x[0])
        if not records:
            continue

        start_date = records[0][0]
        current_start = start_date
        current_end = current_start + timedelta(days=365)
        doses_per_year_real[name] = {}

        year_index = 1
        doses_per_year_real[name][year_index] = 0

        for date, dose in records:
            while date >= current_end:
                year_index += 1
                current_start = current_end
                current_end = current_start + timedelta(days=365)
                doses_per_year_real[name][year_index] = 0

            doses_per_year_real[name][year_index] += dose

    # ترتيب البيانات حسب خيار المستخدم
    sort_option = sort_var.get()
    sorted_data = sorted(filtered, key=lambda x: x[sort_option] if sort_option != "Name" else x["Name"].lower())

    scroll_frame = ctk.CTkScrollableFrame(content_frame, corner_radius=10, fg_color="#ffffff")
    scroll_frame.pack(fill="both", expand=True, padx=20, pady=10)

    headers = ["Select", "Patient Name", "Study ID", "Date", "Modality", "Dose (mSv)", "Total Accumulated Dose", "Latest Year Dose"]
    for col, header in enumerate(headers):
        lbl = ctk.CTkLabel(scroll_frame, text=header, font=ctk.CTkFont(size=14, weight="bold"))
        lbl.grid(row=0, column=col, padx=10, pady=10, sticky="w")

    check_vars.clear()
    for row, data in enumerate(sorted_data, start=1):
        name = data["Name"]
        date = data["Date"]

        # الجرعة التراكمية حسب التاريخ والاسم (حسب القانون الجديد)
        accumulated_dose = accumulated_dose_dict.get((name, date), 0)

        # نحدد آخر سنة فيها فحص للحصول على الجرعة السنوية الأخيرة
        patient_years = doses_per_year_real.get(name, {})
        if patient_years:
            latest_year = max(patient_years.keys())
            latest_year_dose = patient_years[latest_year]
        else:
            latest_year_dose = 0

        var = ctk.BooleanVar(value=data in selected_cases)
        check_vars.append((var, data))
        chk = ctk.CTkCheckBox(scroll_frame, variable=var)
        chk.grid(row=row, column=0, padx=10, pady=5, sticky="w")

        ctk.CTkLabel(scroll_frame, text=name).grid(row=row, column=1, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=str(data["StudyID"])).grid(row=row, column=2, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=date.strftime("%Y-%m-%d")).grid(row=row, column=3, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=data["Modality"]).grid(row=row, column=4, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=f"{data['mSv']:.2f}").grid(row=row, column=5, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=f"{accumulated_dose:.2f}").grid(row=row, column=6, padx=10, pady=5, sticky="w")
        ctk.CTkLabel(scroll_frame, text=f"{latest_year_dose:.2f}").grid(row=row, column=7, padx=10, pady=5, sticky="w")
def update_selected_cases():
    selected_cases.clear()
    for var, data in check_vars:
        if var.get():
            selected_cases.append(data)

def show_hl7_message():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select at least one case to view HL7 message.")
        return

    password = simpledialog.askstring("Password", "Enter password to view HL7 message:", show='*')
    if password != "admin123":
        messagebox.showerror("Unauthorized", "Incorrect password.")
        return

    data = selected_cases[-1]
    hl7_filename = f"{HL7_DIR}/{data['Name']}_{data['Date'].strftime('%Y%m%d')}_{data['StudyID']}.hl7"
    if os.path.exists(hl7_filename):
        with open(hl7_filename, "r") as f:
            hl7 = f.read()
        messagebox.showinfo("HL7 Message", hl7)
    else:
        messagebox.showerror("Not Found", "HL7 message not found.")

def show_selected_cases():
    update_selected_cases()
    if len(selected_cases) not in [2, 4]:
        messagebox.showwarning("Selection Error", "Please select 2 or 4 cases.")
        return

    top = ctk.CTkToplevel(root)
    top.title("Selected Cases Viewer")
    top.geometry("1100x700")
    top.lift()

    for idx, data in enumerate(selected_cases):
        row = idx // 2
        col = idx % 2

        frame = ctk.CTkFrame(top, fg_color="#f0f0f0", corner_radius=10)
        frame.grid(row=row, column=col, padx=20, pady=20, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=10)

        info = (
            f"👤 Name: {data['Name']}\n"
            f"🆔 ID: {data['StudyID']}\n"
            f"📅 Date: {data['Date'].date()}\n"
            f"🧬 Type: {data['Modality']}\n"
            f"☢ Dose: {data['mSv']:.2f} mSv\n"
            f"🧪 CTDIvol: {data['CTDIvol']} mGy\n"
            f"📏 DLP: {data['DLP']} mGy·cm\n"
            f"⚧ Sex: {data['Sex']}\n"
            f"🎂 DOB: {data['DOB']}"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=10)

    for i in range(2):
        top.grid_columnconfigure(i, weight=1)
        top.grid_rowconfigure(i, weight=1)

def delete_selected():
    update_selected_cases()
    if not selected_cases:
        messagebox.showwarning("No Selection", "Please select cases to delete.")
        return
    for sel in selected_cases:
        if sel in all_data:
            all_data.remove(sel)
    selected_cases.clear()
    display_text_data()

def resize_bg(event):
    global bg_img_resized, bg_label
    new_width = event.width
    new_height = event.height
    resized = bg_img_orig.resize((new_width, new_height), Image.ANTIALIAS)
    bg_img_resized = ImageTk.PhotoImage(resized)
    bg_label.configure(image=bg_img_resized)

# واجهة المستخدم
root = ctk.CTk()
root.title("DICOM Viewer - Responsive Design")
root.geometry("1300x900")

bg_img_orig = Image.open("g.jpg")
bg_img_resized = ImageTk.PhotoImage(bg_img_orig)
bg_label = ctk.CTkLabel(root, image=bg_img_resized, text="")
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
root.bind("<Configure>", resize_bg)

# أزرار التحكم
ctk.CTkButton(root, text="Load DICOM Files", command=read_dicom_files, width=150, height=35).place(relx=0.02, rely=0.02)
ctk.CTkButton(root, text="Show HL7 Message", command=show_hl7_message, width=150, height=35).place(relx=0.20, rely=0.02)
ctk.CTkButton(root, text="Show Selected Cases", command=show_selected_cases, width=180, height=35).place(relx=0.37, rely=0.02)
ctk.CTkButton(root, text="Delete Selected", command=delete_selected, width=150, height=35).place(relx=0.55, rely=0.02)

# خيارات الفلترة والترتيب
sort_var = ctk.StringVar(value="Name")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.73, rely=0.015)
ctk.CTkOptionMenu(root, variable=sort_var, values=["Name", "Date"], command=lambda _: display_text_data()).place(relx=0.78, rely=0.015)

name_filter_var = ctk.StringVar()
ctk.CTkEntry(root, placeholder_text="Filter by Name", textvariable=name_filter_var).place(relx=0.02, rely=0.07, relwidth=0.2)
name_filter_var.trace_add("write", lambda *args: display_text_data())

date_filter_var = ctk.StringVar()
ctk.CTkEntry(root, placeholder_text="Filter by Date (YYYY-MM-DD)", textvariable=date_filter_var).place(relx=0.24, rely=0.07, relwidth=0.2)
date_filter_var.trace_add("write", lambda *args: display_text_data())

# محتوى البيانات
content_frame = ctk.CTkFrame(root, fg_color="#ffffff", corner_radius=15)
content_frame.place(relx=0.02, rely=0.13, relwidth=0.96, relheight=0.85)

# رسالة ترحيبية
welcome_label = ctk.CTkLabel(content_frame, text="Click here to select DICOM files",
                              text_color="blue", font=ctk.CTkFont(size=20, weight="bold"), cursor="hand2")
welcome_label.pack(expand=True)
welcome_label.bind("<Button-1>", lambda e: read_dicom_files())

root.mainloop()


In [ ]:
import pydicom
import os
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageTk
import datetime
import csv
from collections import defaultdict
from cryptography.fernet import Fernet

# ثابت: مفتاح تشفير للـ HL7
ENCRYPTION_KEY = Fernet.generate_key()
fernet = Fernet(ENCRYPTION_KEY)

# دالة استخراج البيانات من ملف DICOM
def extract_dicom_info(dicom_file):
    ds = pydicom.dcmread(dicom_file)
    name = ds.PatientName.family_name if 'PatientName' in ds else "Unknown"
    date = ds.StudyDate if 'StudyDate' in ds else "00000000"
    dose = float(ds.get('CTDIvol', 0.0))
    modality = ds.get('Modality', 'NA')
    patient_id = ds.get('PatientID', 'Unknown')
    return {
        "name": name,
        "date": datetime.datetime.strptime(date, "%Y%m%d").date(),
        "dose": dose,
        "modality": modality,
        "id": patient_id,
        "file": dicom_file
    }

# دالة حساب الجرعة لكل سنة لكل مريض
def calculate_dose_per_year(data):
    result = defaultdict(lambda: defaultdict(float))
    for row in data:
        name = row['name']
        year = row['date'].year
        result[name][year] += row['dose']
    return result

# حفظ البيانات في ملف CSV
def save_to_csv(data, filename="rad.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Date', 'Dose', 'Modality', 'ID'])
        for row in data:
            writer.writerow([row['name'], row['date'], row['dose'], row['modality'], row['id']])

# توليد رسالة HL7 مشفرة
def generate_hl7(row):
    hl7_msg = f"PID|1||{row['id']}||{row['name']}|||\n" \
              f"OBR|1|||{row['modality']}|||||{row['date']}\n" \
              f"OBX|1|NM|CTDIvol||{row['dose']}|mGy"
    encrypted = fernet.encrypt(hl7_msg.encode()).decode()
    return encrypted

# فك التشفير
def decrypt_hl7(encrypted_msg, password):
    if password != "admin123":
        return "Wrong password"
    try:
        decrypted = fernet.decrypt(encrypted_msg.encode()).decode()
        return decrypted
    except:
        return "Invalid message"

# واجهة البرنامج
class DICOMViewer:
    def __init__(self, root):
        self.root = root
        self.root.title("DICOM Dose Viewer")
        self.data = []
        self.selected = []

        self.label = tk.Label(root, text="Click here to load DICOM files", fg="blue", font=("Arial", 18, "bold"))
        self.label.pack(pady=10)
        self.label.bind("<Button-1>", self.load_files)

        self.tree = ttk.Treeview(root, columns=("Name", "Date", "Dose", "Modality", "ID"), show="headings")
        for col in self.tree["columns"]:
            self.tree.heading(col, text=col)
        self.tree.pack(padx=10, pady=10, fill="both", expand=True)

        self.tree.bind("<Double-1>", self.on_select)

        self.btn_frame = tk.Frame(root)
        self.btn_frame.pack(pady=10)

        tk.Button(self.btn_frame, text="Show HL7", command=self.show_hl7).pack(side="left", padx=5)
        tk.Button(self.btn_frame, text="Show 2 Cases", command=lambda: self.show_cases(2)).pack(side="left", padx=5)
        tk.Button(self.btn_frame, text="Show 4 Cases", command=lambda: self.show_cases(4)).pack(side="left", padx=5)
        tk.Button(self.btn_frame, text="Save CSV", command=lambda: save_to_csv(self.data)).pack(side="left", padx=5)

    def load_files(self, event=None):
        files = filedialog.askopenfilenames(filetypes=[("DICOM Files", "*.dcm")])
        self.data.clear()
        for file in files:
            info = extract_dicom_info(file)
            self.data.append(info)
        self.tree.delete(*self.tree.get_children())
        for row in self.data:
            self.tree.insert('', 'end', values=(row['name'], row['date'], row['dose'], row['modality'], row['id']))

        save_to_csv(self.data)

        doses = calculate_dose_per_year(self.data)
        print("\n📊 الجرعة السنوية لكل مريض:")
        for name, years in doses.items():
            for year, dose in years.items():
                print(f"{name} - {year}: {dose:.2f} mGy")

    def on_select(self, event):
        item = self.tree.selection()[0]
        values = self.tree.item(item, 'values')
        for row in self.data:
            if str(row['name']) == values[0] and str(row['date']) == values[1]:
                if row not in self.selected:
                    self.selected.append(row)
                break

    def show_hl7(self):
        if not self.selected:
            messagebox.showinfo("Info", "Please select a record by double-clicking")
            return
        row = self.selected[-1]
        encrypted_msg = generate_hl7(row)
        password = tk.simpledialog.askstring("HL7 Password", "Enter password to view HL7:")
        result = decrypt_hl7(encrypted_msg, password)
        messagebox.showinfo("HL7 Message", result)

    def show_cases(self, count):
        if len(self.selected) < count:
            messagebox.showwarning("Warning", f"Select at least {count} cases by double-clicking.")
            return
        top = tk.Toplevel(self.root)
        top.title(f"Showing {count} DICOM Cases")
        for i in range(count):
            row = self.selected[-(i+1)]
            frame = tk.Frame(top, bd=2, relief="solid")
            frame.grid(row=i//2, column=i%2, padx=10, pady=10)
            lbl = tk.Label(frame, text=f"{row['name']}\n{row['date']}\n{row['dose']} mGy\n{row['modality']}", font=("Arial", 10))
            lbl.pack()
            try:
                ds = pydicom.dcmread(row['file'])
                arr = ds.pixel_array
                img = Image.fromarray(arr).resize((200, 200))
                img_tk = ImageTk.PhotoImage(img)
                img_lbl = tk.Label(frame, image=img_tk)
                img_lbl.image = img_tk
                img_lbl.pack()
            except:
                tk.Label(frame, text="Image Unavailable").pack()

# تشغيل البرنامج
root = tk.Tk()
root.geometry("1000x700")
app = DICOMViewer(root)
root.mainloop()
